In [34]:
import stanza

# proxies = {'http': 'http://ip:port', 'https': 'http://ip:port'}
stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

2023-01-10 09:54:57 INFO: Downloading default packages for language: en (English) ...
2023-01-10 09:54:58 INFO: File exists: C:\Users\Yixian\stanza_resources\en\default.zip
2023-01-10 09:55:01 INFO: Finished downloading models and saved to C:\Users\Yixian\stanza_resources.
2023-01-10 09:55:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-10 09:55:01 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-01-10 09:55:02 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2023-01-10 09:55:02 INFO: Use device: gpu
2023-01-10 09:55:02 INFO: Loading: tokenize
2023-01-10 09:55:02 INFO: Loading: pos
2023-01-10 09:55:02 INFO: Loading: lemma
2023-01-10 09:55:02 INFO: Loading: depparse
2023-01-10 09:55:02 INFO: Done loading processors!


In [113]:
sentences = ["My fat dog vocalizes well into my new pink microphone"]

def print_dep(sentences):
    for sentence in sentences:
        doc = nlp(sentence)
        print(*[f'id: {word.id}\tupos: {word.upos}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')

print_dep(sentences)

id: 1	upos: PRON	word: My	head id: 3	head: dog	deprel: nmod:poss
id: 2	upos: NOUN	word: fat	head id: 3	head: dog	deprel: compound
id: 3	upos: NOUN	word: dog	head id: 4	head: vocalizes	deprel: nsubj
id: 4	upos: VERB	word: vocalizes	head id: 0	head: root	deprel: root
id: 5	upos: ADV	word: well	head id: 4	head: vocalizes	deprel: advmod
id: 6	upos: ADP	word: into	head id: 10	head: microphone	deprel: case
id: 7	upos: PRON	word: my	head id: 10	head: microphone	deprel: nmod:poss
id: 8	upos: ADJ	word: new	head id: 10	head: microphone	deprel: amod
id: 9	upos: ADJ	word: pink	head id: 10	head: microphone	deprel: amod
id: 10	upos: NOUN	word: microphone	head id: 4	head: vocalizes	deprel: obl


In [115]:
sentences = ["My fat dog vocalizes well into my new pink microphone"]

def find_adjectives(sentence):
    adjectives = []
    doc = nlp(sentence)
    adjectives = [word.text for sentence in doc.sentences for word in sentence.words if word.upos == "ADJ"]
    return adjectives

def find_triplets(sentence, adjectives=None):

    triplets = []
    nouns = []

    doc = nlp(sentence)

    for sentence in doc.sentences:
        for word in sentence.words:
            head_word_text = sentence.words[word.head-1].text
            if word.upos == "NOUN" and head_word_text in adjectives:
                # Append in the form: (adjective, noun, deprel)
                triplets.append({"ADJ": head_word_text, "NOUN": word.text, "dependency": word.deprel})
                nouns.append(word.text)
            elif word.upos == "NOUN":
                nouns.append(word.text)

    for sentence in doc.sentences:
        for word in sentence.words:
            head_word_text = sentence.words[word.head-1].text
            if word.upos == "ADJ" and head_word_text in nouns:
                # Append in the form: (adjective, noun, deprel)
                triplets.append({"ADJ": word.text, "NOUN": head_word_text, "dependency": word.deprel})

    return triplets, nouns

for sentence in sentences:
    adjectives = find_adjectives(sentence)
    triplets = find_triplets(sentence, adjectives)
    print(adjectives, triplets)


['new', 'pink'] ([{'ADJ': 'new', 'NOUN': 'microphone', 'dependency': 'amod'}, {'ADJ': 'pink', 'NOUN': 'microphone', 'dependency': 'amod'}], ['fat', 'dog', 'microphone'])


In [ ]:
# Find adjectives
# From noun, find head (if adj is after)
# From adj, find head 

In [40]:
nlp(sentences[0])

[
  [
    {
      "id": 1,
      "text": "The",
      "lemma": "the",
      "upos": "DET",
      "xpos": "DT",
      "feats": "Definite=Def|PronType=Art",
      "head": 3,
      "deprel": "det",
      "start_char": 0,
      "end_char": 3
    },
    {
      "id": 2,
      "text": "cheap",
      "lemma": "cheap",
      "upos": "ADJ",
      "xpos": "JJ",
      "feats": "Degree=Pos",
      "head": 3,
      "deprel": "amod",
      "start_char": 4,
      "end_char": 9
    },
    {
      "id": 3,
      "text": "laptop",
      "lemma": "laptop",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "Number=Sing",
      "head": 6,
      "deprel": "nsubj",
      "start_char": 10,
      "end_char": 16
    },
    {
      "id": 4,
      "text": "is",
      "lemma": "be",
      "upos": "AUX",
      "xpos": "VBZ",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
      "head": 6,
      "deprel": "cop",
      "start_char": 17,
      "end_char": 19
    },
    {
      "id": 5,
 

In [3]:
# Extracting adjectives
processed = doc.sentences[0]
adjectives = [word.lemma for word in doc.sentences[0].words if word.upos == "ADJ"]

In [4]:
print(adjectives)

['cheap', 'slow']


## 50 random sentences from IMDB Dataset

In [5]:
from datasets import load_dataset

data_files = {"data": "imdb_dataset.csv"}
imdb_dataset = load_dataset("csv", data_files = data_files)

Using custom data configuration default-3bd7d10a1b0f878f


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating data split: 0 examples [00:00, ? examples/s]

c:\Users\Yixian\miniconda3\envs\machine-learning\lib\site-packages\datasets\download\streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to C:/Users/Yixian/.cache/huggingface/datasets/csv/default-3bd7d10a1b0f878f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
import html

# Random sample
imdb_sample = imdb_dataset["data"].shuffle(seed=5)
imdb_sample["review"]

# Convert html unicode from reviews
imdb_dataset = imdb_dataset.map(lambda x: {"review": html.unescape(x["review"])})

# Remove line breaks
imdb_dataset = imdb_dataset.map(lambda x: {"review": x["review"].replace("<br />", " ")})

# Set all to lowercase
imdb_dataset = imdb_dataset.map(lambda x: {"review": x["review"].lower()})

  0%|          | 0/50000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

In [32]:
sentences = []

# Get all sentences in the dataset
import re

for review in imdb_dataset["data"]["review"]:
    # Split into sentences by either fullstop, question mark or exclamation mark, but ignore ellipses
    sentences.extend(re.split(r'(?<!\.)(?<!\!)(?<!\?)[.!?]', review))

# Ensure that sentences have at least 20 characters
sentences = [sentence for sentence in sentences if len(sentence) >= 20]

# Get 50 random sentences
import random

random_numbers = random.sample(range(0, len(sentences)-1), 50)
random_sentences = [sentences[number] for number in random_numbers]

In [33]:
random_sentences

['this was one of the most moving movies i have ever seen',
 ' in fact, nobody gets killed',
 " in the end freeman offers scarlet little more than strange diversion with a 'star',not even paying for gas for scarlet's dead of night return to her unchanged life in a town the name of which freeman cares not to know",
 ' the whole thing is evocative of a previous age and previous movies but it sweeps away the old and refreshes with a modern tale of redemption amid the tommy-gun shootouts and extortion rackets',
 ' it rips off so many other films (can you say blade',
 " here, those who don't read their lines like cigar-store indians sound like they learned them phonetically",
 ' it all ends well with oil being struck and "fishtail" going to military school',
 "  this isn't exceptionally scary or thrilling but if you have an hour and a half to kill and/or you want to end up feeling frustrated and confused, rent this winner",
 " all of those things are weird, but what's really the strangest t